## Ticker data fetch noteboook
The following notebook is used to fetch ticker data. 

### Env variables and imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import os.path
import sys
import uuid
import copy
import json
import warnings

import pandas as pd
import requests
import tqdm
import backtrader as bt
import pyfolio as pf

warnings.filterwarnings("ignore")  # Avoid some noise

In [ ]:
PROJECT_LOCATION = "/home/narboom23/Projects/licenta"
DATA_LOCATION = f"{PROJECT_LOCATION}/data"
DAILY_TICKER_DATA_LOCATION = f"{DATA_LOCATION}/daily_tickers"
WEEKLY_TICKER_DATA_LOCATION = f"{DATA_LOCATION}/weekly_tickers"

In [ ]:
# All stocks are from SP500 and atleast 22 years - since 1998 - to provide ample space for indicators

TICKER_FULL_LIST = {
    "MMM": ("3M Company", "1976-08-09"),
    "ABT": ("Abbott Laboratories", "1964-03-31"),
    "ADBE": ("Adobe Inc.", "1997-05-05"),
    "AMD": ("Advanced Micro Devices", ""),
    "AES": ("AES Corp", "1998-10-02"),
    "APD": ("Air Products & Chemicals", "1985-04-30"),
    "ALL": ("Allstate Corp", "1995-07-13"),
    "MO": ("Altria Group Inc", "1957-03-04"),
    "AEE": ("Ameren Corp", "1991-09-19"),
    "AEP": ("American Electric Power", "1957-03-04"),
    "AXP": ("American Express", "1976-06-30"),
    "AIG": ("American International Group", "1980-03-31"),
    "AMGN": ("Amgen Inc.", "1992-01-02"),
    "AON": ("Aon plc", "1996-04-23"),
    "APA": ("APA Corporation", "1997-07-28"),
    "AAPL": ("Apple Inc.", "1982-11-30"),
    "AMAT": ("Applied Materials Inc.", "1995-03-16"),
    "ADM": ("Archer-Daniels-Midland Co", "1981-07-29"),
    "T": ("AT&T Inc.", "1983-11-30 (1957-03-04)"),
    "ADSK": ("Autodesk Inc.", "1989-12-01"),
    "ADP": ("Automatic Data Processing", "1981-03-31"),
    "AZO": ("AutoZone Inc", "1997-01-02"),
    "AVY": ("Avery Dennison Corp", "1987-12-31"),
    "BLL": ("Ball Corp", "1984-10-31"),
    "BAC": ("Bank of America Corp", "1976-06-30"),
    "BK": ("The Bank of New York Mellon", "1995-03-31"),
    "BAX": ("Baxter International Inc.", "1972-09-30"),
    "BDX": ("Becton Dickinson", "1972-09-30"),
    "BA": ("Boeing Company", "1957-03-04"),
    "BSX": ("Boston Scientific", "1995-02-24"),
    "BMY": ("Bristol-Myers Squibb", "1957-03-04"),
    "CPB": ("Campbell Soup", "1957-03-04"),
    "COF": ("Capital One Financial", "1998-07-01"),
    "CAH": ("Cardinal Health Inc.", "1997-05-27"),
    "CCL": ("Carnival Corp.", "1998-12-22"),
    "CAT": ("Caterpillar Inc.", "1957-03-04"),
    "CNP": ("CenterPoint Energy", "1985-07-31"),
    "SCHW": ("Charles Schwab Corporation", "1997-06-02"),
    "CVX": ("Chevron Corp.", "1957-03-04"),
    "CI": ("Cigna", "1976-06-30"),
    "CINF": ("Cincinnati Financial", "1997-12-18"),
    "CSCO": ("Cisco Systems", "1993-12-01"),
    "C": ("Citigroup Inc.", "1988-05-31"),
    "CLX": ("The Clorox Company", "1969-03-31"),
    "KO": ("Coca-Cola Company", "1957-03-04"),
    "CL": ("Colgate-Palmolive", "1957-03-04"),
    "CMA": ("Comerica Inc.", "1995-12-01"),
    "CAG": ("Conagra Brands", "1983-08-31"),
    "COP": ("ConocoPhillips", "1957-03-04"),
    "ED": ("Consolidated Edison", ""),
    "GLW": ("Corning Inc.", ""),
    "COST": ("Costco Wholesale Corp.", "1993-10-01"),
    "CSX": ("CSX Corp.", "1967-09-30"),
    "CMI": ("Cummins Inc.", "1965-03-31"),
    "CVS": ("CVS Health", "1957-03-04"),
    "DHR": ("Danaher Corp.", ""),
    "DRI": ("Darden Restaurants", ""),
    "DE": ("Deere & Co.", "1957-03-04"),
    "D": ("Dominion Energy", ""),
    "DOV": ("Dover Corporation", "1985-10-31"),
    "DTE": ("DTE Energy Co.", "1957-03-04"),
    "DUK": ("Duke Energy", "1976-06-30"),
    "EMN": ("Eastman Chemical", "1994-01-01"),
    "ETN": ("Eaton Corporation", ""),
    "ECL": ("Ecolab Inc.", "1989-01-31"),
    "EIX": ("Edison Int'l", "1957-03-04"),
    "EMR": ("Emerson Electric Company", "1965-03-31"),
    "ETR": ("Entergy Corp.", "1957-03-04"),
    "EFX": ("Equifax Inc.", "1997-06-19"),
    "ES": ("Eversource Energy", ""),
    "EXC": ("Exelon Corp.", "1957-03-04"),
    "XOM": ("Exxon Mobil Corp.", "1957-03-04"),
    "FDX": ("FedEx Corporation", "1980-12-31"),
    "FITB": ("Fifth Third Bancorp", ""),
    "FE": ("FirstEnergy Corp", ""),
    "F": ("Ford Motor Company", "1957-03-04"),
    "BEN": ("Franklin Resources", ""),
    "FCX": ("Freeport-McMoRan Inc.", ""),
    "GPS": ("Gap Inc.", "1986-08-31"),
    "GD": ("General Dynamics", "1957-03-04"),
    "GE": ("General Electric", ""),
    "GIS": ("General Mills", "1969-03-31"),
    "GPC": ("Genuine Parts", "1973-12-31"),
    "GL": ("Globe Life Inc.", "1989-04-30"),
    "GWW": ("Grainger (W.W.) Inc.", "1981-06-30"),
    "HAL": ("Halliburton Co.", "1957-03-04"),
    "HIG": ("Hartford Financial Svc.Gp.", "1957-03-04"),
    "HAS": ("Hasbro Inc.", "1984-09-30"),
    "HSY": ("The Hershey Company", "1957-03-04"),
    "HES": ("Hess Corporation", "1984-05-31"),
    "HD": ("Home Depot", "1988-03-31"),
    "HON": ("Honeywell Int'l Inc.", "1964-03-31"),
    "HWM": ("Howmet Aerospace", "1964-03-31"),
    "HPQ": ("HP Inc.", "1974-12-31"),
    "HUM": ("Humana Inc.", ""),
    "HBAN": ("Huntington Bancshares", ""),
    "ITW": ("Illinois Tool Works", "1986-02-28"),
    "INTC": ("Intel Corp.", "1976-12-31"),
    "IBM": ("International Business Machines", "1957-03-04"),
    "IP": ("International Paper", "1957-03-04"),
    "IPG": ("Interpublic Group", "1992-10-01"),
    "IFF": ("International Flavors & Fragrances", "1976-03-31"),
    "JNJ": ("Johnson & Johnson", "1973-06-30"),
    "JPM": ("JPMorgan Chase & Co.", "1975-06-30"),
    "K": ("Kellogg Co.", ""),
    "KEY": ("KeyCorp", "1994-03-01"),
    "KMB": ("Kimberly-Clark", "1957-03-04"),
    "KLAC": ("KLA Corporation", ""),
    "KR": ("Kroger Co.", "1957-03-04"),
    "LB": ("L Brands Inc.", "1983-09-30"),
    "LEG": ("Leggett & Platt", ""),
    "LLY": ("Lilly (Eli) & Co.", "1970-12-31"),
    "LNC": ("Lincoln National", "1976-06-30"),
    "LIN": ("Linde plc", "1992-07-01"),
    "LMT": ("Lockheed Martin Corp.", "1984-07-31"),
    "L": ("Loews Corp.", ""),
    "LOW": ("Lowe's Cos.", "1984-02-29"),
    "MRO": ("Marathon Oil Corp.", "1991-05-01"),
    "MAR": ("Marriott International", ""),
    "MMC": ("Marsh & McLennan", "1987-08-31"),
    "MAS": ("Masco Corp.", "1981-06-30"),
    "MKC": ("McCormick & Co.", ""),
    "MCD": ("McDonald's Corp.", "1970-06-30"),
    "MCK": ("McKesson Corp.", ""),
    "MDT": ("Medtronic plc", "1986-10-31"),
    "MRK": ("Merck & Co.", "1957-03-04"),
    "MU": ("Micron Technology", "1994-09-27"),
    "MSFT": ("Microsoft Corp.", "1994-06-01"),
    "TAP": ("Molson Coors Beverage Company", "1976-06-30"),
    "MCO": ("Moody's Corp", ""),
    "MS": ("Morgan Stanley", ""),
    "MSI": ("Motorola Solutions Inc.", ""),
    "NWL": ("Newell Brands", "1989-04-30"),
    "NEM": ("Newmont Corporation", "1969-06-30"),
    "NEE": ("NextEra Energy", "1976-06-30"),
    "NKE": ("Nike, Inc.", "1988-11-30"),
    "NI": ("NiSource Inc.", ""),
    "NSC": ("Norfolk Southern Corp.", "1957-03-04"),
    "NTRS": ("Northern Trust Corp.", ""),
    "NOC": ("Northrop Grumman", "1985-06-30"),
    "NUE": ("Nucor Corp.", "1985-04-30"),
    "OXY": ("Occidental Petroleum", "1982-12-31"),
    "OMC": ("Omnicom Group", ""),
    "ORCL": ("Oracle Corp.", "1989-08-31"),
    "PCAR": ("Paccar", "1980-12-31"),
    "PH": ("Parker-Hannifin", "1985-11-30"),
    "PAYX": ("Paychex Inc.", ""),
    "PEP": ("PepsiCo Inc.", "1957-03-04"),
    "PKI": ("PerkinElmer", "1985-05-31"),
    "PFE": ("Pfizer Inc.", "1957-03-04"),
    "PNW": ("Pinnacle West Capital", ""),
    "PNC": ("PNC Financial Services", "1988-04-30"),
    "PPG": ("PPG Industries", "1957-03-04"),
    "PPL": ("PPL Corp.", ""),
    "PG": ("Procter & Gamble", "1957-03-04"),
    "PGR": ("Progressive Corp.", "1997-08-04"),
    "PEG": ("Public Service Enterprise Group (PSEG)", "1957-03-04"),
    "PHM": ("PulteGroup", "1984-04-30"),
    "QCOM": ("Qualcomm", ""),
    "RTX": ("Raytheon Technologies", ""),
    "RF": ("Regions Financial Corp.", "1998-08-28"),
    "ROK": ("Rockwell Automation Inc.", ""),
    "SPGI": ("S&P Global Inc.", ""),
    "SLB": ("Schlumberger Ltd.", "1965-03-31"),
    "SHW": ("Sherwin-Williams", "1964-06-30"),
    "SNA": ("Snap-on", "1982-09-30"),
    "SO": ("Southern Company", "1957-03-04"),
    "LUV": ("Southwest Airlines", "1994-07-01"),
    "SWK": ("Stanley Black & Decker", "1982-09-30"),
    "SBUX": ("Starbucks Corp.", ""),
    "STT": ("State Street Corp.", ""),
    "TGT": ("Target Corp.", "1976-12-31"),
    "TXN": ("Texas Instruments", ""),
    "TXT": ("Textron Inc.", "1978-12-31"),
    "TJX": ("TJX Companies Inc.", "1985-09-30"),
    "TFC": ("Truist Financial", "1997-12-04"),
    "TSN": ("Tyson Foods", ""),
    "USB": ("U.S. Bancorp", ""),
    "UNP": ("Union Pacific Corp", "1957-03-04"),
    "UNH": ("UnitedHealth Group Inc.", "1994-07-01"),
    "UNM": ("Unum Group", "1994-03-01"),
    "VLO": ("Valero Energy", ""),
    "VZ": ("Verizon Communications", "1983-11-30"),
    "VFC": ("VF Corporation", "1979-06-30"),
    "VNO": ("Vornado Realty Trust", ""),
    "WMT": ("Walmart", "1982-08-31"),
    "WBA": ("Walgreens Boots Alliance", "1979-12-31"),
    "DIS": ("The Walt Disney Company", "1976-06-30"),
    "WM": ("Waste Management Inc.", ""),
    "WAT": ("Waters Corporation", ""),
    "WFC": ("Wells Fargo", "1976-06-30"),
    "WY": ("Weyerhaeuser", ""),
    "WHR": ("Whirlpool Corp.", ""),
    "WMB": ("Williams Companies", "1975-03-31"),
    "XEL": ("Xcel Energy Inc", "1957-03-04"),
    "YUM": ("Yum! Brands Inc", "1997-10-06"),
}
TICKERS = list(TICKER_FULL_LIST.keys())

### Part 1 - Daily ticker data from yahoo finance

In [ ]:
len(TICKERS)

In [ ]:
def add_entry_to_csv(csv_filename, trade_info_dict):
    csv_path = f"{RESULTS_LOCATION}/{csv_filename}.csv"

    trade_info_dict = {k: [v] for k, v in trade_info_dict.items()}

    trade_df = pd.DataFrame.from_dict(trade_info_dict)
    trade_df.to_csv(csv_path, mode="a", index=False, header=False)

In [ ]:
def get_ticker_csv_path(ticker_name):
    return f"{DAILY_TICKER_DATA_LOCATION}/{ticker_name}.csv"


def get_ticker_csv_as_df(ticker_name):
    return pd.read_csv(get_ticker_csv_path(ticker_name))

In [ ]:
def download_latest_ticker_csv(ticker_name):
    url = "https://query1.finance.yahoo.com/v7/finance/download/{TICKER_NAME}?period1=883616400&period2=1609459199&interval=1d&events=history&includeAdjustedClose=true"
    # 1609459199 - 2020.12.31 11:59:59 PM GMT
    # 946688400 - 2000.01.01 01:00:00 AM GMT
    # 883616400 - Thursday, January 1, 1998 1:00:00 AM

    yahoo_url = url.format(TICKER_NAME=ticker_name)
    file_respone = requests.get(yahoo_url)

    csv_path = get_ticker_csv_path(ticker_name)

    with open(csv_path, "wb") as f:
        f.write(file_respone.content)


def update_all_ticker_csvs():
    with tqdm.tqdm(total=len(TICKERS)) as pbar:
        for ticker_name in TICKERS:
            download_latest_ticker_csv(ticker_name)
            pbar.update(1)


update_all_ticker_csvs()
# download_latest_ticker_csv('GOOG')

In [ ]:
datalist = [
    (get_ticker_csv_path(ticker_name), ticker_name)
    for ticker_name in list(TICKER_FULL_LIST.keys())
]


for fl, t in datalist:
    try:
        with open(fl, "r") as f:
            if len(f.readlines()) < 5000:
                print(t)
    except:
        print("Error ", t)

### Part 2 - Weekly ticker data from yahoo finance

In [ ]:
def add_entry_to_csv(csv_filename, trade_info_dict):
    csv_path = f"{RESULTS_LOCATION}/{csv_filename}.csv"

    trade_info_dict = {k: [v] for k, v in trade_info_dict.items()}

    trade_df = pd.DataFrame.from_dict(trade_info_dict)
    trade_df.to_csv(csv_path, mode="a", index=False, header=False)

In [ ]:
def get_ticker_csv_path(ticker_name):
    return f"{WEEKLY_TICKER_DATA_LOCATION}/{ticker_name}.csv"


def get_ticker_csv_as_df(ticker_name):
    return pd.read_csv(get_ticker_csv_path(ticker_name))

In [ ]:
def download_latest_ticker_csv(ticker_name):
    url = "https://query1.finance.yahoo.com/v7/finance/download/{TICKER_NAME}?period1=883616400&period2=1609459199&interval=1wk&events=history&includeAdjustedClose=true"
    # The 'interval' is different!

    # 1609459199 - 2020.12.31 11:59:59 PM GMT
    # 946688400 - 2000.01.01 01:00:00 AM GMT
    # 883616400 - Thursday, January 1, 1998 1:00:00 AM

    yahoo_url = url.format(TICKER_NAME=ticker_name)
    file_respone = requests.get(yahoo_url)

    csv_path = get_ticker_csv_path(ticker_name)

    with open(csv_path, "wb") as f:
        f.write(file_respone.content)


def update_all_ticker_csvs():
    with tqdm.tqdm(total=len(TICKERS)) as pbar:
        for ticker_name in TICKERS:
            download_latest_ticker_csv(ticker_name)
            pbar.update(1)


update_all_ticker_csvs()
# download_latest_ticker_csv('GOOG')